In [1]:

from sklearn.base import TransformerMixin, BaseEstimator

from collections import Counter , defaultdict

import pandas as pd

pd.set_option('display.max_columns', None)


from pandas import Series as s , DataFrame as df
import numpy as np

import seaborn as sns
# import matplotlib.pyplot as plt

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt, rcParams as rc


%matplotlib inline
rc["figure.figsize"] = 10,6

import datetime
import datetime as dt

from sklearn.model_selection  import StratifiedKFold
from sklearn.decomposition import PCA

from iteration_utilities import duplicates, unique_everseen

import sys
from itertools import groupby
from operator import itemgetter
from timeit import timeit

In [44]:

#Algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.naive_bayes import MultinomialNB

from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier

#preprocessing
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import train_test_split

import lightgbm as lgb


#Metrics
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, recall_score, precision_score
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_recall_fscore_support

from sklearn.pipeline import Pipeline

# GridSearchCV to find optimal min_samples_leaf
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv("train_Wc8LBpr.csv")

df_test = pd.read_csv("test_VsU9xXK.csv")
submission = pd.read_csv("sample_submission_NoPBkjr.csv")

df10 = df.copy()
df.shape, df_test.shape, submission.shape

((131662, 14), (87395, 13), (87395, 2))

In [5]:
def get_percentage_miss_value(dataset):
    higher_miss_value_column = []
    miss_threshold_value = 50
    
    for i in dataset.columns:
        if dataset[i].isna().sum() > 1: 
            perectange_val = (dataset[i].isna().sum() / len(dataset)) * 100
            print("Column-> " , i, ", total no of missing value : ",dataset[i].isna().sum() , " & :         ", round(perectange_val,2) ," %")
                
            if(perectange_val > miss_threshold_value):
                higher_miss_value_column.append(i)
            
    print("\n\n\n")
    
    if higher_miss_value_column:
        print("Higher Missing values in Columns for Delete : ", higher_miss_value_column)
    else:
        print("There are no Higher Column Missing values in Dataset")

In [6]:
# get_percentage_miss_value(df)

In [7]:
# get_percentage_miss_value(df_test)

In [8]:
# df.info()

In [9]:
def check_cloumn_details_type_categorical(dataset):
    for i in dataset.columns:
        if(dataset[i].dtype == "object"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            
            print("*"*100)
            print("\n")

In [10]:
# check_cloumn_details_type_categorical(df)

In [11]:
93900 / len(df)

0.7131898345764154

In [12]:
# check_cloumn_details_type_categorical(df_test)

In [13]:
def check_cloumn_details_type_numberical(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "int"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            
            
def check_cloumn_details_type_float(dataset):
    for i in dataset.columns:
        if (dataset[i].dtype == "float"):
            print("Columns name :  ",i  )
            
            print(dict(Counter(dataset[i])))
            print("*"*100)
            print("\n")
            

In [14]:
# check_cloumn_details_type_numberical(df)

In [15]:
# check_cloumn_details_type_float(df)

In [16]:
# Visulazing the distibution of the data for every feature
def visualize_hist(dataset):
    dataset.hist(edgecolor='black', linewidth=1.2, figsize=(20, 20));
    plt.show()

In [17]:
# visualize_hist(df)

In [18]:
def visualize_histogram(dataset):
    # plot histogram
    plt.figure(figsize=(25, 9))  # figure size in ratio 16:9
    features = dataset.columns  # list of columns name
    for i, j in enumerate(features):
        plt.subplot(3, 3, i + 1)  # create subplot for histogram
        plt.title("Histogram of {}".format(j), fontsize=15)  # title of histogram

        bins = len(dataset[j].unique())  # bins for histogram
        plt.hist(dataset[j], bins=bins, rwidth=0.8, edgecolor="y", linewidth=2, )  # plot histogram

    plt.subplots_adjust(hspace=0.5)  # space between horixontal axes (subplots)

In [19]:
# visualize_histogram(df)

In [20]:

    
def corr_metrix(dataset):
    corr = dataset.corr()
    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap='RdYlGn')

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});
    ax.tick_params(labelsize=20)

def corr_2_more_visualize(dataset):
    corr = dataset.corr()
    
    f, ax = plt.subplots(figsize=(20, 9))
    sns.heatmap(corr.apply(lambda x : np.round(x,2)), 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,annot=True,cmap='RdYlGn', annot_kws={"size": 15})
    ax.tick_params(labelsize=20)
    plt.show()
    
    
def linear_regression_train_test():
    #Linear regression with L2 regularization
    for i in range(-2, 3):
        alpha = 10**i
        rm = Ridge(alpha = alpha)
        ridge_model = rm.fit(X_train, y_train)
        preds_ridge = ridge_model.predict(X_test)
    
        plt.scatter(preds_ridge, y_test, alpha= 0.75, c= 'b')
        plt.xlabel('Predicted price')
        plt.ylabel('Actual price')
        plt.title('Ridge redularization with alpha {}'.format(alpha))
        overlay = 'R square: {} \nRMSE: {}'.format(ridge_model.score(X_test, y_test), mean_squared_error(y_test, preds_ridge))
        plt.annotate(s = overlay, xy = (12.1, 10.6), size = 'x-large')
        plt.show()
        
def actual_predict_visualization(actual_values, predict_values):
    plt.scatter(actual_values, predict_values, alpha= 0.75, color = 'b')

    plt.xlabel('Predicted price')
    plt.ylabel('Actual price')
    plt.title('Regression Model')
    plt.show()
    
def visualize_original_predict_test(X_val, Y_val, default = "Test"):
    print("Dataset type  : \n" ,default)
    plt.scatter(diabetes_X_test, diabetes_y_test,  color='black')
    plt.plot(diabetes_X_test, diabetes_y_pred, color='blue', linewidth=3)

    plt.xticks(())
    plt.yticks(())
    
    plt.xlabel("X axis", color = "red")
    plt.ylabel("Y axis", color = "green")

    plt.show()
    
    
def visualize_categorical_values(dataset):
    no_of_columns = 4
    no_of_rows = 4
    
    columns_object_type = [i for i in dataset.columns  if dataset[i].dtype == "object"]
    total_rows = (len(columns_object_type) // no_of_rows ) + 1
    
    f, axes = plt.subplots(total_rows, no_of_columns, figsize=(18,24))

    for ind, val in enumerate(columns_object_type):
        sns.countplot(df[val] , ax = axes[ind // no_of_rows , ind %no_of_columns ])
    plt.show()   
    
sns.pairplot(data = telecom_df[['arpu_6','arpu_7','arpu_8','churn_flag']],hue = 'churn_flag')

def visualize_pairplot_target(numberical_col_list , target_val):
    plt.figure(figsize=(18,34))
    sns.pairplot(data = telecom_df[numberical_col_list],hue = target_val)
    
    #     sns.pairplot(data = telecom_df[['arpu_6','arpu_7','arpu_8','churn_flag']],hue = 'churn_flag') 
    #check for classification
    plt.show()
    
    
def visualize_numberical_values(dataset):
    plt.figure(figsize=(18,34))
    sns.pairplot(df)
    plt.show()
# visualize_numberical_values(df)
    
def check_skewness_numerical(dataset, target):
    #analysing the distribution of sale price
    print('skew is', dataset.SalePrice.skew())   
    plt.hist(dataset[target], color= 'b')

    plt.title('Distribution of ' + target, fontsize = 24)
    plt.ylabel('observation', fontsize = 20)
    plt.xlabel(target, fontsize = 20)

    plt.show()
    
def skewness_after_log_transform(dataset , target):
    #log transforming sale price to transform it into gaussian distribution
    target1 = np.log(dataset.target)
    print('skew is', target1.skew())
    plt.hist(target, color= 'b')

    plt.title('Distribution of ' + target, fontsize = 24)
    plt.ylabel('observation', fontsize = 20)
    plt.xlabel(target , fontsize = 20)

    plt.show()
    
# Since pairplots for all numeric values together are not clear we can make groups ,do plot with price & analyse

def pair_plot(list_4_numberical_values):
    sns.pairplot(df2, x_vars= list_4_numberical_values, y_vars='SalePrice',size=4, kind='scatter')
    plt.show()

NameError: name 'telecom_df' is not defined

In [21]:
# corr_2_more_visualize(df)

In [22]:
# visualize_categorical_values(df)

In [23]:
# visualize_numberical_values(df)

In [24]:
#Label encoding
def convert_to_numerical_datatype_train(dataset):
    enc = LabelEncoder()
    for i in dataset.columns:
        if(dataset[i].dtype == "object"):
            dataset[i] = enc.fit_transform(dataset[i])
            
    return dataset

In [25]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0] if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
    


# <font color ='green' > EDA

In [26]:
drop_col = ["Trip_ID" , "Var1"]

In [27]:
df1 = df.drop(columns = drop_col)
df.shape , df1.shape

((131662, 14), (131662, 12))

In [28]:
df1_test = df_test.drop(columns = drop_col)
df_test.shape , df1_test.shape

((87395, 13), (87395, 11))

In [29]:
df2= DataFrameImputer().fit_transform(df1)


df2_test = DataFrameImputer().fit_transform(df1_test)

In [30]:
df2.isna().sum().sum(), df2_test.isna().sum().sum()

(0, 0)

In [31]:
df3 = convert_to_numerical_datatype_train(df2)
df2.shape, df3.shape

((131662, 12), (131662, 12))

In [32]:
df3_test = convert_to_numerical_datatype_train(df2_test)
df2_test.shape, df3_test.shape

((87395, 11), (87395, 11))

In [33]:
X = df3.iloc[:,:-1]

Y = df3.iloc[:,-1]

In [34]:
def standardscaler_preprocessing(dataset_train, dataset_test, num_col):
    scaler = StandardScaler()
   
    dataset_train[num_col] = scaler.fit_transform(dataset_train[num_col])

    dataset_test[num_col] = scaler.transform(dataset_test[num_col])
    
    return dataset_train, dataset_test

In [66]:
def get_accuracy(y_train_val , y_pred_val , dataset_type = "Default"):
    
    print(" Dataset type is : ", dataset_type)
    
    print("\n Accuracy Score     : ",round(accuracy_score(y_train_val, y_pred_val), 4) * 100)
    
    print("\n precision_accuracy : ",round(precision_score(y_train_val, y_pred_val), 4) * 100)
 
    print("\n recall_accuracy    : ",round(recall_score(y_train_val, y_pred_val), 4) * 100)
    
    print("\n roc_auc_accuracy   : ",round(roc_auc_score(y_train_val, y_pred_val), 4) * 100)
    
    print("\n f1_score_accuracy  : ",round(f1_score(y_train_val, y_pred_val), 4) * 100)
    
    print("\n explained_variance  : ",round(explained_variance_score(y_train_val, y_pred_val), 4) * 100)
    
    tn, fp, fn, tp = confusion_matrix(y_train_val, y_pred_val).ravel()
    
    print("\n Confusion Matrix TN : ", tn, " FP : ", fp, " FN : ", fn, " TP : ", tp)

In [67]:

rows=[]

def addRandomStateForAlgorithm(x,y,names,algorithms,columns_name,random_state_list):    
    for j in range(len(algorithms)):
        model = algorithms[j]
        for i in random_state_list:
            
            x_train, x_test , y_train , y_test = train_test_split(x ,y , test_size = 0.30 , random_state = i)
            
            model.fit(x_train,y_train)
            
            y_pred_train = model.predict(x_train)
            y_pred = model.predict(x_test)
            
            train_acc = accuracy_score(y_train, y_pred_train)
            train_acc = round(train_acc, 4) * 100
            
            test_acc = accuracy_score(y_test, y_pred)
            test_acc = round(test_acc, 4) * 100
            
#             roc_auc_score_acc = roc_auc_score(y_test, y_pred)
#             roc_auc_score_acc = round(roc_auc_score_acc, 4) * 100
            

#             row = [names[j],   i,   train_acc, test_acc, roc_auc_score_acc]
            row = [names[j],   i,   train_acc, test_acc]
    
            rows.append(row)
            
    models_df = pd.DataFrame(rows) 
    
    models_df.columns = columns_name
    print(models_df)
    

In [68]:
   
# names_regression = [ "LightGBM", "RF", "XGBoost" , "SVM"]
# algorithms = [ LGBMClassifier(), RandomForestClassifier(), XGBClassifier(), SVC()]

# columns_name = ["Model",    "Random_state",   'Train_acc',     "Test_acc" , "roc_auc_score"]

names_regression = [ "XGBClassifier"]
algorithms = [ XGBClassifier(max_depth=5, objective='multi:softmax', num_classes=3)]

columns_name = ["Model",    "Random_state",   'Train_acc',     "Test_acc" ]


random_state_list_up_to_10 = [1]


addRandomStateForAlgorithm(X,Y,names_regression,algorithms,columns_name,random_state_list_up_to_10)

           Model  Random_state  Train_acc  Test_acc
0  XGBClassifier             1      69.48     68.88


In [88]:
X_train, X_test, y_train, y_test = train_test_split(X ,Y , test_size = 0.05 , random_state = 10)



lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':4,
    'metric': 'multi_logloss',
    'learning_rate': 0.002296,
    'max_depth': 7,
    'num_leaves': 17,
    'feature_fraction': 0.4,
    'bagging_fraction': 0.6,
    'bagging_freq': 17}





gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)






y_pred_final = gbm.predict(df3_test, num_iteration=gbm.best_iteration)

[1]	valid_0's multi_logloss: 1.05301
Training until validation scores don't improve for 5 rounds
[2]	valid_0's multi_logloss: 1.05261
[3]	valid_0's multi_logloss: 1.05197
[4]	valid_0's multi_logloss: 1.0513
[5]	valid_0's multi_logloss: 1.05037
[6]	valid_0's multi_logloss: 1.05002
[7]	valid_0's multi_logloss: 1.04963
[8]	valid_0's multi_logloss: 1.049
[9]	valid_0's multi_logloss: 1.04884
[10]	valid_0's multi_logloss: 1.04843
[11]	valid_0's multi_logloss: 1.04806
[12]	valid_0's multi_logloss: 1.04795
[13]	valid_0's multi_logloss: 1.04727
[14]	valid_0's multi_logloss: 1.04685
[15]	valid_0's multi_logloss: 1.04652
[16]	valid_0's multi_logloss: 1.04607
[17]	valid_0's multi_logloss: 1.04545
[18]	valid_0's multi_logloss: 1.04509
[19]	valid_0's multi_logloss: 1.04465
[20]	valid_0's multi_logloss: 1.04451
Did not meet early stopping. Best iteration is:
[20]	valid_0's multi_logloss: 1.04451


In [83]:
# model_final = XGBClassifier(
#  learning_rate =0.1,
#  n_estimators=1000,
#  max_depth=5,
#  min_child_weight=1,
#  gamma=0,
#  subsample=0.8,
#  colsample_bytree=0.8,
#  objective= 'binary:logistic',
#  nthread=4,
#  scale_pos_weight=1,
#  seed=27)

# model_final = XGBClassifier(max_depth=5, objective='multi:softmax', num_classes=3)


# model_final.fit(X, Y)

# y_pred_final = model_final.predict(df3_test)

In [89]:
submission_1 = submission.copy()

print(df_test.shape , submission.shape , len(y_pred_final))


(87395, 13) (87395, 2) 87395


In [90]:
submission_1["Surge_Pricing_Type"] = y_pred_final

print(submission_1["Surge_Pricing_Type"].values)

[1.00369591e-15 1.00006445e-15 9.99272686e-16 ... 1.00393520e-15
 9.99706876e-16 9.99606503e-16]


In [91]:



submission_1.to_csv('submission_7_LightGBM.csv', index=False)

df_submission_1 = pd.read_csv('submission_7_LightGBM.csv')


In [92]:
print(df_submission_1.shape)

df_submission_1.head(3)

(87395, 2)


,Trip_ID,Surge_Pricing_Type
0,T0005689459,1.003696e-15
1,T0005689462,1.000064e-15
2,T0005689463,9.992727e-16
